# Battle of neighborhoods

In [404]:
import numpy as np
import pandas as pd
import requests
from sklearn import preprocessing
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

## Reading data

In [405]:
# Reading neighbors data from csv file
df = pd.read_csv("tarragona_neighborhoods.csv")
df.head() 

,Postalcode,Neighborhood,City,Population,Latitude,Longitude
0,43002,Nou Eixample Nord,Tarragona,26080,41.122781,1.247971
1,43006,Torreforta,Tarragona,17010,41.118823,1.216954
2,43007,Sant Pere i Sant Pau,Tarragona,16694,41.138271,1.252438
3,43005,Nou Eixample Sud,Tarragona,15884,41.116315,1.245997
4,43007,Urbanitzacions de Llevant,Tarragona,13152,41.118099,1.272726


## Retrieving venues from Foursquare API

Foursquare API to calculate the number of restaurants for each neighborhood. Data needed to calculate the ratio population/restaurant and also to create the clustering. 

In [406]:
CLIENT_ID = 'RB5EO3FB3IR4BOCM0OFBLPATPXNWDJ1PUUTKWOQOFKUP1AAY' # your Foursquare ID
CLIENT_SECRET = 'FFYDHMRFTE4WYFUW5FGHBXF3KZ0J3POEP23WB10FEXJNJWBH' # your Foursquare Secret
VERSION = '20180604'

In [407]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venues', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [408]:
venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Nou Eixample Nord
Torreforta
Sant Pere i Sant Pau
Nou Eixample Sud
Urbanitzacions de Llevant
Campclar
Eixample Tarragona
Barris Marítims
Bonavista
Sant Salvador
Part Alta
Districte 1
Districte 2
Districte 3
Districte 4
Districte 5
Districte 6
Districte 7
Districte 8
Districte 9
Districte 10
El Vendrell
Cambrils
Salou
Valls
Calafell
Vila-seca


In [409]:
venues.set_index('Neighborhood')
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venues,Venue Latitude,Venue Longitude,Venue Category
0,Nou Eixample Nord,41.122781,1.247971,Pizza Nova,41.118857,1.246968,Pizza Place
1,Nou Eixample Nord,41.122781,1.247971,Tradicionarius,41.120784,1.251550,Bakery
2,Nou Eixample Nord,41.122781,1.247971,Plaça de la Imperial Tàrraco,41.118683,1.245670,Plaza
3,Nou Eixample Nord,41.122781,1.247971,Casa Boada,41.119042,1.250761,Sandwich Place
4,Nou Eixample Nord,41.122781,1.247971,Panishop,41.120327,1.246174,Coffee Shop


## Analyzing data

In [410]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venues,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Barris Marítims,19,19,19,19,19,19
Bonavista,8,8,8,8,8,8
Calafell,4,4,4,4,4,4
Cambrils,70,70,70,70,70,70
Campclar,6,6,6,6,6,6
Districte 1,93,93,93,93,93,93
Districte 10,4,4,4,4,4,4
Districte 2,16,16,16,16,16,16
Districte 3,10,10,10,10,10,10


In [411]:
# Adding columns 'Venues' per neighborhood
venues_per_neighborhood = pd.DataFrame(venues.groupby('Neighborhood').count()['Venues'])
df = df.join(venues_per_neighborhood, on='Neighborhood')
df.head()

,Postalcode,Neighborhood,City,Population,Latitude,Longitude,Venues
0,43002,Nou Eixample Nord,Tarragona,26080,41.122781,1.247971,12
1,43006,Torreforta,Tarragona,17010,41.118823,1.216954,5
2,43007,Sant Pere i Sant Pau,Tarragona,16694,41.138271,1.252438,6
3,43005,Nou Eixample Sud,Tarragona,15884,41.116315,1.245997,42
4,43007,Urbanitzacions de Llevant,Tarragona,13152,41.118099,1.272726,8


In [412]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
onehot.insert(loc=0, column='Neighborhood', value=venues['Neighborhood'] )
onehot.shape

(664, 101)

In [413]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped.head()

,Neighborhood,Accessories Store,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,...,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Theme Park,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store
0,Barris Marítims,0.000000,0.0,0.0,0.0,0.052632,0.000000,0.0,0.052632,0.000000,...,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,Bonavista,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.125000,0.000000,...,0.0,0.125000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,Calafell,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Cambrils,0.014286,0.0,0.0,0.0,0.000000,0.014286,0.0,0.014286,0.014286,...,0.0,0.085714,0.0,0.0,0.0,0.0,0.0,0.014286,0.0,0.0
4,Campclar,0.000000,0.0,0.0,0.0,0.166667,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [414]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [415]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barris Marítims,Seafood Restaurant,Mediterranean Restaurant,Japanese Restaurant,Bar,Park,Pool,Athletics & Sports,Ice Cream Shop,Tapas Restaurant,Supermarket
1,Bonavista,Spanish Restaurant,Restaurant,Soccer Stadium,Farmers Market,Bar,Tapas Restaurant,Gym,Soccer Field,Diner,Donut Shop
2,Calafell,Lawyer,Park,Fruit & Vegetable Store,Mediterranean Restaurant,Women's Store,Food & Drink Shop,Diner,Donut Shop,Electronics Store,Event Space
3,Cambrils,Mediterranean Restaurant,Seafood Restaurant,Tapas Restaurant,Ice Cream Shop,Italian Restaurant,Spanish Restaurant,Pizza Place,Hotel,Café,Restaurant
4,Campclar,Supermarket,Gym / Fitness Center,Athletics & Sports,Pharmacy,Soccer Field,Women's Store,Food,Dessert Shop,Diner,Donut Shop


# Making Clusters for Neighborhood

Using Clustering to Subdivide Data

In [416]:
# set number of clusters
kclusters = 10

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 1, 6, 1, 9, 1, 1, 1])

In [417]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = df

# merge grouped with df to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head()

,Postalcode,Neighborhood,City,Population,Latitude,Longitude,Venues,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43002,Nou Eixample Nord,Tarragona,26080,41.122781,1.247971,12,1,Pizza Place,Coffee Shop,Breakfast Spot,Plaza,Sandwich Place,Café,Burger Joint,Food & Drink Shop,Bakery,Hotel
1,43006,Torreforta,Tarragona,17010,41.118823,1.216954,5,1,Department Store,Hotel,Asian Restaurant,Supermarket,Stadium,Food & Drink Shop,Dessert Shop,Diner,Donut Shop,Electronics Store
2,43007,Sant Pere i Sant Pau,Tarragona,16694,41.138271,1.252438,6,7,Bakery,Spanish Restaurant,Brewery,Soccer Field,Women's Store,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market
3,43005,Nou Eixample Sud,Tarragona,15884,41.116315,1.245997,42,1,Hotel,Supermarket,Pizza Place,Bakery,Bar,Clothing Store,Plaza,Nightclub,Coffee Shop,Tapas Restaurant
4,43007,Urbanitzacions de Llevant,Tarragona,13152,41.118099,1.272726,8,1,Spanish Restaurant,Restaurant,Beach Bar,Supermarket,Beach,Breakfast Spot,Hotel,Electronics Store,Event Space,French Restaurant


In [418]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Barris Marítims,Seafood Restaurant,Mediterranean Restaurant,Japanese Restaurant,Bar,Park,Pool,Athletics & Sports,Ice Cream Shop,Tapas Restaurant,Supermarket
1,1,Bonavista,Spanish Restaurant,Restaurant,Soccer Stadium,Farmers Market,Bar,Tapas Restaurant,Gym,Soccer Field,Diner,Donut Shop
2,0,Calafell,Lawyer,Park,Fruit & Vegetable Store,Mediterranean Restaurant,Women's Store,Food & Drink Shop,Diner,Donut Shop,Electronics Store,Event Space
3,1,Cambrils,Mediterranean Restaurant,Seafood Restaurant,Tapas Restaurant,Ice Cream Shop,Italian Restaurant,Spanish Restaurant,Pizza Place,Hotel,Café,Restaurant
4,6,Campclar,Supermarket,Gym / Fitness Center,Athletics & Sports,Pharmacy,Soccer Field,Women's Store,Food,Dessert Shop,Diner,Donut Shop


In [419]:
from geopy.geocoders import Nominatim 

address = 'Tarragona, ES'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tarragona are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tarragona are 41.1172364, 1.2546057.


#### Creating Map

In [420]:
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Selecting neighborhoods similar to Reus District 1

In [421]:
cluster_label = merged.loc[merged['Neighborhood'] == 'Nou Eixample Sud']['Cluster Labels'].values[0]
print('Reus District 1 is assigned to cluster number', cluster_label)

Reus District 1 is assigned to cluster number 1


In [422]:
# Selecting neighborhoods similiar to Reus District 1
selected_neighborhoods = merged.loc[merged['Cluster Labels'] == cluster_label]
selected_neighborhoods.head()

,Postalcode,Neighborhood,City,Population,Latitude,Longitude,Venues,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43002,Nou Eixample Nord,Tarragona,26080,41.122781,1.247971,12,1,Pizza Place,Coffee Shop,Breakfast Spot,Plaza,Sandwich Place,Café,Burger Joint,Food & Drink Shop,Bakery,Hotel
1,43006,Torreforta,Tarragona,17010,41.118823,1.216954,5,1,Department Store,Hotel,Asian Restaurant,Supermarket,Stadium,Food & Drink Shop,Dessert Shop,Diner,Donut Shop,Electronics Store
3,43005,Nou Eixample Sud,Tarragona,15884,41.116315,1.245997,42,1,Hotel,Supermarket,Pizza Place,Bakery,Bar,Clothing Store,Plaza,Nightclub,Coffee Shop,Tapas Restaurant
4,43007,Urbanitzacions de Llevant,Tarragona,13152,41.118099,1.272726,8,1,Spanish Restaurant,Restaurant,Beach Bar,Supermarket,Beach,Breakfast Spot,Hotel,Electronics Store,Event Space,French Restaurant
6,43001,Eixample Tarragona,Tarragona,11333,41.115150,1.251046,100,1,Mediterranean Restaurant,Ice Cream Shop,Bar,Plaza,Cocktail Bar,Supermarket,Tapas Restaurant,Burger Joint,Restaurant,Nightclub


# Getting the right neighborhood

In [423]:
# Filtering df by cluster 1
print(df.shape)
df = df[df.set_index(['Neighborhood']).index.isin(selected_neighborhoods.set_index(['Neighborhood']).index)]
print(df.shape)

(27, 7)
(18, 7)


In [424]:
# Calculating venues per population ratio
df['Target'] = df['Population']/df['Venues']
df.head()

,Postalcode,Neighborhood,City,Population,Latitude,Longitude,Venues,Target
0,43002,Nou Eixample Nord,Tarragona,26080,41.122781,1.247971,12,2173.333333
1,43006,Torreforta,Tarragona,17010,41.118823,1.216954,5,3402.000000
3,43005,Nou Eixample Sud,Tarragona,15884,41.116315,1.245997,42,378.190476
4,43007,Urbanitzacions de Llevant,Tarragona,13152,41.118099,1.272726,8,1644.000000
6,43001,Eixample Tarragona,Tarragona,11333,41.115150,1.251046,100,113.330000


In [425]:
# The 5 best neighborhoods
df.nlargest(5, 'Target')

,Postalcode,Neighborhood,City,Population,Latitude,Longitude,Venues,Target
21,43700,El Vendrell,El Vendrell,35821,41.217300,1.530048,9,3980.111111
1,43006,Torreforta,Tarragona,17010,41.118823,1.216954,5,3402.000000
0,43002,Nou Eixample Nord,Tarragona,26080,41.122781,1.247971,12,2173.333333
13,43205,Districte 3,Reus,18686,41.145110,1.107541,10,1868.600000
4,43007,Urbanitzacions de Llevant,Tarragona,13152,41.118099,1.272726,8,1644.000000


In [426]:
# Getting the neighborhood with maximum target
best_neighborhood=df.nlargest(1, 'Target')
print('The best neighborhood is', best_neighborhood['Neighborhood'].values[0],'(',best_neighborhood['City'].values[0],')')

The best neighborhood is El Vendrell ( El Vendrell )
